In [1]:
#Import H2O libiraries
import h2o
from h2o.automl import H2OAutoML

In [2]:
#Initialize local H2O Cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /Users/thomasott/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/9c/tqhnrz3x207bf8pzjxjhkm100000gn/T/tmpe7cd2hgj
  JVM stdout: /var/folders/9c/tqhnrz3x207bf8pzjxjhkm100000gn/T/tmpe7cd2hgj/h2o_thomasott_started_from_python.out
  JVM stderr: /var/folders/9c/tqhnrz3x207bf8pzjxjhkm100000gn/T/tmpe7cd2hgj/h2o_thomasott_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.3
H2O cluster version age:,7 months and 10 days !!!
H2O cluster name:,H2O_from_python_thomasott_xvp6ua
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
#Load in Loan Loss Training Set
train = h2o.import_file('./data/train_v2.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
# Set target and predictor variables
y = "loss"
x = train.col_names
x.remove(y)
x.remove("id")

In [5]:
#Call H2O-3 AutoML. Set early stopping metric to MAE and CV Folds = 5
aml = H2OAutoML(max_models = 20, seed=1234, stopping_metric = "MAE", sort_metric = "MAE", nfolds = 5)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [6]:
#Example LeaderBoard (Note: my small CPU can only train a single model, run this on a larger machine!)
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20191217_082049,18.6435,4.31782,18.6435,1.42045,0.72176


In [7]:
#Import Driverless AI transformed features from oringal training Set
train = h2o.import_file('./data/train_v2.zip.1575494585.3482676.bin.munged_train.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
#Check out the munged dataset
train

306_f39,380_f46,407_f493,783_ClusterDist50:f170:f517:f766.5,783_ClusterDist50:f170:f517:f766.6,783_ClusterDist50:f170:f517:f766.7,783_ClusterDist50:f170:f517:f766.9,783_ClusterDist50:f170:f517:f766.10,loss
0.65705,0.70127,19.72,4.18907,3.46873,4.04611,3.43021,0,0
0.7471,0.80819,124,3.04902,3.70761,2.21696,3.53115,2.47696,0
0.77405,0.8207,903,5.23919,3.15662,1.08105,5.94185,4.17258,0
0.78385,0.86382,130.94,4.65057,2.7428,1.28935,5.13865,3.25539,0
0.79085,0.82485,399,1.9061,4.80431,3.01405,2.78507,2.98162,0
0.7269,0.89431,836.75,3.71168,3.10235,2.00807,4.07024,2.66102,1
0.76075,0.92268,252.92,2.8794,4.33658,2.02854,3.86323,3.37263,0
0.7995,0.88271,82,4.05847,3.72011,1.46525,4.88991,3.85273,0
0.7868,0.84464,82.5,4.26453,2.37536,2.40069,4.26683,2.21446,0
0.74885,0.80162,399,4.31491,2.28665,2.83093,4.097,1.89826,0


In [10]:
# Set target and predictor variables
y = "loss"
x = train.col_names
x.remove(y)

In [12]:
#Run another AutoML with munged data set
aml_munge = H2OAutoML(max_models = 20, seed=1234, stopping_metric = "MAE", sort_metric = "MAE", nfolds = 5)
aml_munge.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [13]:
#check out top models (note the munged data set makes training faster)
lb = aml_munge.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
DeepLearning_grid_1_AutoML_20191217_092436_model_2,19.7147,4.44012,19.7147,1.03178,nan
DeepLearning_grid_1_AutoML_20191217_092436_model_1,19.6976,4.43819,19.6976,1.05991,nan
DeepLearning_1_AutoML_20191217_092436,19.4548,4.41076,19.4548,1.25196,nan
XGBoost_grid_1_AutoML_20191217_092436_model_1,19.3854,4.40289,19.3854,1.26134,0.659004
XGBoost_3_AutoML_20191217_092436,19.354,4.39932,19.354,1.39207,0.705898
XGBoost_2_AutoML_20191217_092436,19.5139,4.41745,19.5139,1.39248,0.724708
XGBoost_1_AutoML_20191217_092436,19.4913,4.4149,19.4913,1.4231,0.727983
GBM_grid_1_AutoML_20191217_092436_model_3,19.3288,4.39645,19.3288,1.44458,0.724869
StackedEnsemble_AllModels_AutoML_20191217_092436,19.329,4.39647,19.329,1.44486,0.725021
GBM_grid_1_AutoML_20191217_092436_model_1,19.3282,4.39638,19.3282,1.44492,0.724951


In [14]:
h2o.cluster().shutdown()

H2O session _sid_92c3 closed.
